In [1]:
import numpy
import pandas
import sklearn
from matplotlib import pyplot

In [2]:
spectra = pandas.read_csv('ondrejov-dataset.csv', index_col='id')
spectra.info()
spectra.head()

<class 'pandas.core.frame.DataFrame'>
Index: 12936 entries, a201305230017 to we200016
Columns: 151 entries, dec to 6732.0
dtypes: float64(142), int64(2), object(7)
memory usage: 15.0+ MB


,dec,gratang,detector,expval,object,dichmir,chipid,ra,specfilt,label,...,6718.20863309,6719.74100719,6721.27338129,6722.8057554,6724.3381295,6725.8705036,6727.4028777,6728.9352518,6730.4676259,6732.0
id,,,,,,,,,,,,,,,,,,,,,
a201305230017,-11:09:40.70,30.25,PyLoN 2048x512BX,19.246130,alp Vir,2,E2V 42-10 BX,13:25:11.580,1,absorption,...,506096.210748,505286.047228,504106.925201,502043.387652,499827.958541,498385.165956,498190.044968,499302.148265,500607.698358,500899.192757
a201306060015,+19:10:56.60,30.25,PyLoN 2048x512BX,0.009187,alp Boo,2,E2V 42-10 BX,14:15:39.667,1,absorption,...,193644.604516,194951.353066,200427.247481,203353.778155,203940.311357,203613.661065,202053.255088,202003.060090,203675.565714,204506.290588
a201306060016,+19:10:56.60,30.25,PyLoN 2048x512BX,0.012332,alp Boo,2,E2V 42-10 BX,14:15:39.667,1,absorption,...,307559.413555,309575.702337,318283.859189,323097.056161,324141.074458,323543.630765,320970.267304,320887.844222,323502.907113,324655.685273
a201306060018,+33:21:45.60,30.25,PyLoN 2048x512BX,5.056532,beta Lyr,2,E2V 42-10 BX,18:50:04.787,1,emission,...,44950.487947,45019.574545,44910.009841,44719.100229,44689.037403,44724.791337,44700.993767,44705.082297,44752.548428,44734.531766
a201306060019,+33:21:45.60,30.25,PyLoN 2048x512BX,15.040130,beta Lyr,2,E2V 42-10 BX,18:50:04.787,1,emission,...,125795.778619,126004.868747,125788.287910,125227.841525,125145.712758,125330.650720,125332.395056,125338.057872,125376.976633,125252.358114
